# Predicting reactions

## Intro
Predicting reactions based on reaction templates is important for synthetic biology, biotechnology, and other areas that need to establish connections between molecules that are not currently annotated in existing databases.
Examples of existing tools for application of reaction templates are:
* [BNICE](https://doi.org/10.1093/bioinformatics/bti213)
* [Pickaxe](https://github.com/tyo-nu/MINE-Database)

The basic mechanism of action is as follows:
1. Take template (e.g. R-C=O -> R-C-OH)
2. Take compound (e.g. C-C-C-C=O)
3. Apply template to compound and get the reaction (e.g. C-C-C-C=O -> C-C-C-C-OH)

A common way to represent reaction templates is [reaction SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html). SMARTS serve as a substructure search linear molecule expression (analagous to regular expressions in text).

One of the existing python libraries for application of reaction SMARTS to compounds for reaction generation is [rdkit](https://www.rdkit.org/docs/source/rdkit.Chem.rdChemReactions.html).

## Predicting reactions using Rhea templates

This notebook generates SMARTS from generic Rhea reactions (reactions with R-compounds or star(\*)-compounds) and applies them as reactions.

1. Use RheaDB class to access the RheaDB data 

In [ ]:
from pyrheadb.RheaDB import RheaDB
rdb = RheaDB()

2. Use ReactionSmartsConverter class to identify Rhea reactions that can act as templates (class reactions) and convert them to SMARTS using atom mapping with [RXNMapper](https://github.com/rxn4chemistry/rxnmapper)

In [ ]:
from pyrheadb.ReactionSmartsConverter import ReactionSmartsConverter
rxn_smarts_cvrt = ReactionSmartsConverter(rdb)

3. Use ReactionPrediction class to apply the resulting reaction SMARTS to desired compounds

In [ ]:
from pyrheadb.ReactionPrediction import ReactionPrediction

In [ ]:
# Initiate ReactionPrediction class - do not forget to initiate it with RheaDB to be able to access Rhea data from the class
rxnpred = ReactionPrediction(rdb)

There are two possibilities to interpret reaction SMARTS:
1. All compounds are included into SMARTS. SMARTS will try to find a pattern match in provided compounds for every necessary reactant, including fully defined cofactors/cosubstrates (e.g. NADH, ATP, etc.)
2. Only use star (*/R) compounds in SMARTS. The default compounds will be added after product structure calculation for balance

* set_star_smarts_only: False - option 1 (reaction SMARTS applied as a whole)
* set_star_smarts_only: True - option 2 (only the star-compounds used)

**First, do the analysis with the full SMARTS**

In [ ]:
rxnpred.set_star_smarts_only(False)

Now lets load SMARTS data that was generated with ReactionSmartsConverter

In [ ]:
rxnpred.load_smarts_data()

In [ ]:
print('Total SMARTS:', len(rxnpred.smarts_data))

In [ ]:
# Uncommit if you want to see all the smarts_data
#rxnpred.smarts_data

In [ ]:
print('Example SMARTS:', rxnpred.smarts_data['39023'])

**4. Predict products and reactions**

You can supply predict_products with:
* SMILES string of one compound. Example: "CCC(=O)[O-]"
* list of SMILES of compounds that are substrates of a single reaction and are reacting with each other. Example: \["CC=O", "NH3"]

From the practical point of view, enumerating several substrates for one reaction is required to generate permutations. This operation increases computational time to evaluate all options.
From the theoretical point of view, it is unlikely that more than two random molecules meet and react.
It was nicely put in words on [Reddit](https://www.reddit.com/r/chemistry/comments/18smdjd/are_there_any_chemical_reactions_that_actually/?rdt=50008): "It’s unlikely for 3 things to bounce into each other at the exact same time in the correct orientation. There are a lot of multi-component reactions but it usually happens in stages."

More on Molecularity: [Wikipedia](https://en.m.wikipedia.org/wiki/Molecularity)

Here we demonstrate how to test all Rhea reaction templates on one molecule and evaluate which products and which reactions we can get.

**Predict products**

In [ ]:
rheaid2products=rxnpred.predict_products('CCC(=O)[O-]')

In [ ]:
product_sets = [i for i in rheaid2products if i[1]]
print('Number of possible product sets that were obtained:',len(product_sets))

In [ ]:
product_sets

**Predict reactions**

In [ ]:
df_pred_rxns=rxnpred.predict_reactions('CCC(=O)[O-]')

In [ ]:
df_pred_rxns

Check smarts for one of the Rhea reactions (18969) which resulted in products for the selected compound

In [ ]:
rxnpred.smarts_data['18969']

**Now, repeat with the star-only SMARTS (default option)**

In [ ]:
rxnpred_only_star = ReactionPrediction(rdb)
rxnpred_only_star.load_smarts_data()
print('Total SMARTS:', len(rxnpred_only_star.smarts_data))

In [ ]:
rheaid_star_only2products=rxnpred_only_star.predict_products('CCC(=O)[O-]')

In [ ]:
product_sets = [i for i in rheaid_star_only2products if i[1]]
print('Number of possible product sets that were obtained:', len(product_sets))

In [ ]:
df_pred_rxns_only_star=rxnpred_only_star.predict_reactions('CCC(=O)[O-]')

In [ ]:
df_pred_rxns_only_star

As you can see, there are more reactions generated with ReactionPrediction.set_star_smarts_only(True). This is because we did not have to add cofactors into the "mix" and all the defined cofactors were automatically found and applied to generate valid balanced reactions!

Therefore, _star_smarts_only = True_ is the default option. However, we kept the ability to use full SMARTS by resetting this option to False as was shown in the beginning.

**Group reactions based on International chemical identifier for reactions (RInChI).**

[Link to article on RInChI](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0277-8)

In [ ]:
df_grouped_by_rinchikey = rxnpred_only_star.group_predicted_reactions_based_on_rinchikey(df_pred_rxns_only_star)

In [ ]:
df_grouped_by_rinchikey

The number of unique reactions descreased and Rhea reaction identifiers are grouped for the same Web-RInChIKey.

# Analysis of the reaction templates

Above we showed application of the library to predict reactions and products for a single compound. Using pyrheadb.ReactionPrediction it is possible to make several compounds fitting a reaction template to react.

Here we are showing the distribution of the number of templates per Rhea reaction (of ~ 3 thousand reactions with at least one R/star-compound).

The number of templates should be x2 of the number of Rhea reactions with R group, since they need to be tried as forward and reverse.

In [ ]:
def count_substrate_templates(rxnpredobj):
    counts_reactants = []
    counts_products = []
    counts_together = []
    for rheaid, rxn in rxnpredobj.rdkit_stereo_rxn_data.items():
        num_reactants = rxn.GetNumReactantTemplates()
        num_products = rxn.GetNumProductTemplates()
        if num_reactants>3 or num_products>3:
            print('4+ substrate templates in', rheaid)
        counts_products.append(num_products)
        counts_reactants.append(num_reactants)
        counts_together.extend([num_reactants, num_products])
    return counts_reactants, counts_products, counts_together

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Data
def plot_bar(elements):

    # Count the occurrences of each category
    counter = Counter(elements)

    # Ensure all categories are present
    categories = ['1', '2', '3', '4+']
    counts = [counter[1], counter[2], counter[3], sum(v for k, v in counter.items() if k >= 4)]

    # Plot
    plt.figure(figsize=(10, 6))
    bars = plt.bar(categories, counts, color='skyblue')

    # Add counts on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(height), ha='center', va='bottom')

    plt.xlabel('Number of substrate templates')
    plt.ylabel('Number of Rhea class reactions (R/star-reactions)')
    plt.title('Number of Reaction Templates Distribution by Number of Substrate Templates')
    plt.show()

**WITHOUT defined cofactors included into SMARTS**

In [ ]:
counts_reactants_1, counts_products_1, counts_together_1 = count_substrate_templates(rxnpred_only_star)

In [ ]:
rxnpred_only_star.smarts_data['30239']

In [ ]:
plot_bar(counts_together_1)

In [ ]:
# Commit two other versions of the plot since they do not add much info
#plot_bar(counts_reactants_1)

In [ ]:
#plot_bar(counts_products_1)

**WITH defined cofactors included into SMARTS**

In [ ]:
counts_reactants_2, counts_products_2, counts_together_2 = count_substrate_templates(rxnpred)

In [ ]:
plot_bar(counts_together_2)

Comparing only-star templates with all-compound templates, we see, that treating defined cofactors separately brings advantage as it reduces the need for combinatorial increase in calculations due to the requirement to have permutations.

Most of the template reaction can be treated as singe substrate template (single SMILES string input to predict_reactions() and predict_products())